In [1]:
import os

os.environ["SPARK_HOME"]="/Users/omar/Desktop/Spark"
os.environ["PYSPARK_PYTHON"] = "python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = ("jupyter"
                                       "")
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "lab"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

print(spark.version)
print(spark.sparkContext._jvm.scala.util.Properties.versionString())

25/04/21 17:05:07 WARN Utils: Your hostname, MacBook-Pro-3.local resolves to a loopback address: 127.0.0.1; using 10.71.0.59 instead (on interface en0)
25/04/21 17:05:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/21 17:05:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.5.5
version 2.12.18


In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("KafkaSparkDemo")
         .master("local[*]")
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5")
         .getOrCreate())


Error: Could not find or load main class org.apache.spark.launcher.Main
Caused by: java.lang.ClassNotFoundException: org.apache.spark.launcher.Main
/Users/omar/Desktop/Spark_Tut/.pyspark-env/lib/python3.13/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [5]:
print(spark.version)

3.5.5


In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("KafkaSparkDemo")
         .master("local[*]")
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0")
         .getOrCreate())

# Use the new PLAINTEXT_HOST listener port (29092)
kafka_df = (spark.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "localhost:29092")  # Changed from 7092 to 29092
      .option("subscribe", "prometheus_metrics")  # Using your existing topic
      .option("startingOffsets", "earliest")
      .load())

# Process the data
parsed_df = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")





Error: Could not find or load main class org.apache.spark.launcher.Main
Caused by: java.lang.ClassNotFoundException: org.apache.spark.launcher.Main
/Users/omar/Desktop/Spark_Tut/.pyspark-env/lib/python3.13/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [12]:
parsed_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [13]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, from_json

labels_schema = StructType([
    StructField("__name__", StringType(), True),
    StructField("instance", StringType(), True),
    StructField("job", StringType(), True),
    StructField("os_type", StringType(), True),
    StructField("vm", StringType(), True)
])

json_schema = StructType([
    StructField("labels", labels_schema, True),
    StructField("name", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("value", StringType(), True)
])

# Kafka `value` is in binary, so cast to string first
parsed_df = kafka_df.selectExpr("CAST(value AS STRING) as json_str")

# Parse JSON
json_df = parsed_df.select(from_json(col("json_str"), json_schema).alias("data"))

# Flatten nested fields
flattened_df = json_df.select(
    col("data.name"),
    col("data.timestamp"),
    col("data.value"),
    col("data.labels.__name__").alias("metric_name"),
    col("data.labels.instance"),
    col("data.labels.job"),
    col("data.labels.os_type"),
    col("data.labels.vm")
)
query = flattened_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .start()

query.awaitTermination()

-------------------------------------------
Batch: 781
-------------------------------------------
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                            |
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NULL|{"labels":{"__name__":"node_cpu_guest_seconds_total","cpu":"0","instance":"10.71.0.59:9103","job":"node","mode":"nice","os_type":"linux","vm":"Lubun

25/04/21 12:04:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/w9/ddv5pgfx77ndmgb8xq_y1lbc0000gp/T/temporary-58ff5313-6e9f-4b21-abd5-6eb0aa81b2ab. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/21 12:04:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/21 12:04:57 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 786
-------------------------------------------
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                   |
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NULL|{"labels":{"__name__":"node_memory_MemFree_bytes","instance":"10.71.0.59:9101","job":"node","os_type":"linux","vm":"Lubuntu"},"name":"node_memory_MemFree_bytes","timestamp":"2

-------------------------------------------
Batch: 787
-------------------------------------------
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                            |
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NULL|{"labels":{"__name__":"node_cpu_guest_seconds_total","cpu":"0","instance":"10.71.0.59:9103","job":"node","mode":"nice","os_type":"linux","vm":"Lubun

25/04/21 12:05:08 ERROR Executor: Exception in task 0.0 in stage 786.0 (TID 786)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:71)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:391)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda/0x000000b001e5b6e8.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1875)
	at java.base/java.io.ObjectOutputStream.write(ObjectOutputStream.java:725)
	at o

StreamingQueryException: [STREAM_FAILED] Query [id = a71f0b34-df46-4d2e-b35f-3ca732b4ab90, runId = ee4d0c0d-f317-4880-a0e3-ce4c43d36dd6] terminated with exception: Job aborted due to stage failure: Task 0 in stage 786.0 failed 1 times, most recent failure: Lost task 0.0 in stage 786.0 (TID 786) (10.71.0.59 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:71)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:391)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda/0x000000b001e5b6e8.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByte

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/omar/Desktop/Spark_Tut/.pyspark-env/lib/python3.13/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/omar/Desktop/Spark_Tut/.pyspark-env/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/omar/Desktop/Spark_Tut/.pyspark-env/lib/python3.13/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
        "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving
